In [1]:
# This works. Processes videos in np arrays, which we can save to folder or use directly

import cv2, os
print(cv2.__version__)
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision
import pandas as pd 
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image

from IPython.core.display import Image, display

import numpy as np
import shutil

3.4.2


In [10]:
!ls '../../videos/9.27_lazy_overhead'

DJI_0026_lr_frame_timestamps.csv
DJI_0026_lr.MP4
DJI_0027_lr_frame_timestamps.csv
DJI_0027_lr.MP4
DJI_0028_lr.MP4
DJI_0029_lr.MP4
DJI_0030_lr.MP4
DJI_0031_lr.MP4
DJI_0033_lr.MP4
DJI_0034_lr.MP4
DJI_0035_lr.MP4
DJI_0036_lr.MP4
DJI_0037_lr.MP4
DJI_0040_lr.MP4
DJI_0041_lr.MP4
DJI_0042_lr.MP4
DJI_0043_lr.MP4
DJIFlightRecord_2018-09-27_[16-20-07]-TxtLogToCsv.csv
DJIFlightRecord_2018-09-30_[08-28-21]-TxtLogToCsv.csv
DJIFlightRecord_2018-09-30_[08-35-09]-TxtLogToCsv.csv
DJIFlightRecord_2018-09-30_[09-58-56]-TxtLogToCsv.csv
DJIFlightRecord_2018-09-30_[10-12-16]-TxtLogToCsv.csv
DJIFlightRecord_2018-09-30_[10-24-27]-TxtLogToCsv.csv
DJIFlightRecord_2018-10-11_[14-58-09]-TxtLogToCsv.csv
DJIFlightRecord_2018-10-11_[15-11-24]-TxtLogToCsv.csv
DJIFlightRecord_2018-10-11_[19-01-09]-TxtLogToCsv.csv
DJIFlightRecord_2018-10-11_[19-13-25]-TxtLogToCsv.csv
DJIFlightRecord_2018-10-11_[19-26-04]-TxtLogToCsv.csv
DJIFlightRecord_2018-10-12_[16-44-29]-TxtLogToCsv.csv
DJIFlightRecord_2018-10-12_[16-57-54]-TxtLogTo

In [32]:
! rm ../../videos/DJI_0022.MP4

In [10]:
#! sudo rm -r '../../videos/9.27_lazy_overhead'

In [3]:
def format_ix(ix): # Takes in int and returns str padded w 0s. Want to keep files in order. Good up to 99k.
    return ('0000'+str(ix))[-5:]

## Folder structure

-videos
    -group_folder 
        vid1.mp4
        vid2.mp4
        vid3.mp4
        actions1.csv
        actions2.csv
        actions3.csv
        -frames
           -vid1
               frame_1
               frame_2
           -vid2
               frame_1
               frame_2
           -vid3
               frame_1
               frame_2
               frame_3
               
Dataset will be given the videos/group_folder/frames directory, which contains the frames of all vids

In [5]:
group_folder = "9.27_lazy_overhead" # this is main folder holds videos, csvs for group

In [16]:
# Move vidfolder from bucket to instance

os.system('sudo gsutil cp -r gs://bean-warehouse/'+group_folder + ' ../../videos')

0

In [11]:
# Remove entire folder if necessary

shutil.rmtree("../../videos/9.27_lazy_overhead/frames")

In [12]:
# Getting folder structure in order

group_folder_path = "../../videos/"+group_folder
frames_path = group_folder_path +"/frames/"
os.system('sudo mkdir '+frames_path)
#vid_ids = ["DJI_0026_lr", "DJI_0027_lr"] 

In [23]:
import glob

vid_paths = glob.glob(group_folder_path+"/*.MP4"); print(vid_paths)
vid_ids = [p[32:43] for p in vid_paths]; vid_ids # lazy

['../../videos/9.27_lazy_overhead/DJI_0026_lr.MP4', '../../videos/9.27_lazy_overhead/DJI_0035_lr.MP4', '../../videos/9.27_lazy_overhead/DJI_0031_lr.MP4', '../../videos/9.27_lazy_overhead/DJI_0043_lr.MP4', '../../videos/9.27_lazy_overhead/DJI_0028_lr.MP4', '../../videos/9.27_lazy_overhead/DJI_0036_lr.MP4', '../../videos/9.27_lazy_overhead/DJI_0037_lr.MP4', '../../videos/9.27_lazy_overhead/DJI_0034_lr.MP4', '../../videos/9.27_lazy_overhead/DJI_0029_lr.MP4', '../../videos/9.27_lazy_overhead/DJI_0041_lr.MP4', '../../videos/9.27_lazy_overhead/DJI_0030_lr.MP4', '../../videos/9.27_lazy_overhead/DJI_0027_lr.MP4', '../../videos/9.27_lazy_overhead/DJI_0033_lr.MP4', '../../videos/9.27_lazy_overhead/DJI_0042_lr.MP4', '../../videos/9.27_lazy_overhead/DJI_0040_lr.MP4']


['DJI_0026_lr',
 'DJI_0035_lr',
 'DJI_0031_lr',
 'DJI_0043_lr',
 'DJI_0028_lr',
 'DJI_0036_lr',
 'DJI_0037_lr',
 'DJI_0034_lr',
 'DJI_0029_lr',
 'DJI_0041_lr',
 'DJI_0030_lr',
 'DJI_0027_lr',
 'DJI_0033_lr',
 'DJI_0042_lr',
 'DJI_0040_lr']

In [7]:
group_folder_path

'../../videos/9.27_lazy_overhead'

In [9]:
"""
vidcap = cv2.VideoCapture('../../videos/9.27_lazy_overhead/DJI_0043_lr.MP4')
success, image = vidcap.read()
num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT)); num_frames"""

19779

In [24]:
IGNORE_THRESHOLD = 900

for vid_id in vid_ids:
    
    # Make folder to hold frames
    vid_frames_path = frames_path + vid_id; print(vid_frames_path)
    try: 
        os.mkdir(vid_frames_path)
    except:
        print("failed to mkdir, removing and making afresh")
        shutil.rmtree(vid_frames_path)
        os.mkdir(vid_frames_path)

    # Grab video, make reader and grab first frame
    vid = group_folder_path +'/'+ vid_id + ".MP4"; print(vid)
    vidcap = cv2.VideoCapture(vid)
    num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    #success,image = vidcap.read() # image is np array

    # Grab timestamp info, and first timestamp
    #first_ts = vidcap.get(cv2.CAP_PROP_POS_MSEC)
    timestamps = []

    #count = 0
    #success = True
    #while success:
    for frame_ix in range(num_frames):
                
        success,image = vidcap.read() # image is np array
        
        #ignore first and last part of video to remove strange landing and takeoff frames
        if (frame_ix < IGNORE_THRESHOLD) or (frame_ix > (num_frames - IGNORE_THRESHOLD)): continue
            
        # Otherwise, write img and timestamp out
        img_path = vid_frames_path +'/frame_'+ format_ix(frame_ix) +".jpg";
        cv2.imwrite(img_path, image)     # save frame as JPEG file
        timestamps.append(vidcap.get(cv2.CAP_PROP_POS_MSEC))
        #count += 1
        
    pd_ts = pd.DataFrame({"timestamp":np.round(timestamps, 0).astype(int)}); print(pd_ts.head())
    pd_ts.to_csv(group_folder_path+'/'+ vid_id+"_frame_timestamps.csv", index=False)

../../videos/9.27_lazy_overhead/frames/DJI_0026_lr
failed to mkdir, removing and making afresh
../../videos/9.27_lazy_overhead/DJI_0026_lr.MP4
   timestamp
0      30063
1      30097
2      30130
3      30163
4      30197
../../videos/9.27_lazy_overhead/frames/DJI_0035_lr
../../videos/9.27_lazy_overhead/DJI_0035_lr.MP4
   timestamp
0      30063
1      30097
2      30130
3      30163
4      30197
../../videos/9.27_lazy_overhead/frames/DJI_0031_lr
../../videos/9.27_lazy_overhead/DJI_0031_lr.MP4
   timestamp
0      30063
1      30097
2      30130
3      30163
4      30197
../../videos/9.27_lazy_overhead/frames/DJI_0043_lr
../../videos/9.27_lazy_overhead/DJI_0043_lr.MP4
   timestamp
0      30063
1      30097
2      30130
3      30163
4      30197
../../videos/9.27_lazy_overhead/frames/DJI_0028_lr
../../videos/9.27_lazy_overhead/DJI_0028_lr.MP4
   timestamp
0      30063
1      30097
2      30130
3      30163
4      30197
../../videos/9.27_lazy_overhead/frames/DJI_0036_lr
../../videos/9.27_la

In [9]:
!ls "../../videos/9.27_lazy_overhead/frames/DJI_0026_lr"

frame_00900.jpg  frame_04454.jpg  frame_08008.jpg  frame_11562.jpg
frame_00901.jpg  frame_04455.jpg  frame_08009.jpg  frame_11563.jpg
frame_00902.jpg  frame_04456.jpg  frame_08010.jpg  frame_11564.jpg
frame_00903.jpg  frame_04457.jpg  frame_08011.jpg  frame_11565.jpg
frame_00904.jpg  frame_04458.jpg  frame_08012.jpg  frame_11566.jpg
frame_00905.jpg  frame_04459.jpg  frame_08013.jpg  frame_11567.jpg
frame_00906.jpg  frame_04460.jpg  frame_08014.jpg  frame_11568.jpg
frame_00907.jpg  frame_04461.jpg  frame_08015.jpg  frame_11569.jpg
frame_00908.jpg  frame_04462.jpg  frame_08016.jpg  frame_11570.jpg
frame_00909.jpg  frame_04463.jpg  frame_08017.jpg  frame_11571.jpg
frame_00910.jpg  frame_04464.jpg  frame_08018.jpg  frame_11572.jpg
frame_00911.jpg  frame_04465.jpg  frame_08019.jpg  frame_11573.jpg
frame_00912.jpg  frame_04466.jpg  frame_08020.jpg  frame_11574.jpg
frame_00913.jpg  frame_04467.jpg  frame_08021.jpg  frame_11575.jpg
frame_00914.jpg  frame_04468.jpg  frame_08022.jpg  frame_11576

In [37]:
#
transform = transforms.Compose([
    transforms.Resize((128)),
    #transforms.RandomCrop(64, pad_if_needed=True),
    transforms.CenterCrop(128),
    transforms.ToTensor(), 
])

# Obnoxious. Images have to be within folder within root
dataset = datasets.ImageFolder(root='./drone', transform=transform)
len(dataset.imgs)

2969

In [39]:
a = torch.randn(1,2); 


tensor([[-2.9886, -0.2219]])

In [0]:
sample = dataset[2968][0].data.cpu()
save_image(sample.data.cpu(), 'sample_image.png')
display(Image('sample_image.png', width=300, unconfined=True))
display(Image('drone/train/frame2968.jpg', width=300, unconfined=True))

In [0]:
# Stitching frames back together into video

img1 = cv2.imread('drone/train/frame100.jpg') # this is a np array
height , width , layers =  img1.shape
fps = 30

video = cv2.VideoWriter('drone.avi', cv2.VideoWriter_fourcc(*"MJPG"), fps, (width,height))

for i in range(count-1):
    img = cv2.imread('drone/train/frame'+str(i)+'.jpg')
    video.write(img)
    
cv2.destroyAllWindows()
video.release()

In [19]:
img1.shape

(720, 1280, 3)

In [0]:
from google.colab import files

files.download('drone.avi')  # from colab to browser download

In [0]:
# Stitching frames back together into video

img1 = cv2.imread('drone/train/frame100.jpg')
img2 = cv2.imread('drone/train/frame101.jpg')
img3 = cv2.imread('drone/train/frame102.jpg')

height , width , layers =  img1.shape

video = cv2.VideoWriter('video.avi', cv2.VideoWriter_fourcc(*"MJPG"),1,(width,height))

video.write(img1)
video.write(img2)
video.write(img3)

cv2.destroyAllWindows()
video.release()

writer = cv2.VideoWriter("output.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30,(640,480))
for frame in range(10):
    writer.write(np.random.randint(0, 255, (480,640,3)).astype('uint8'))
writer.release()

In [4]:
# Dowload youtube-dl

!sudo curl -L https://yt-dl.org/downloads/latest/youtube-dl -o /usr/local/bin/youtube-dl
!sudo chmod a+rx /usr/local/bin/youtube-dl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     3    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     3    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100   599    0   599    0     0    446      0 --:--:--  0:00:01 --:--:--   446
100 1655k  100 1655k    0     0   477k      0  0:00:03  0:00:03 --:--:-- 1027k


In [5]:
# download clip
!youtube-dl https://www.youtube.com/watch?v=bR4Gq9qfpnM

[youtube] bR4Gq9qfpnM: Downloading webpage
[youtube] bR4Gq9qfpnM: Downloading video info webpage
[download] Destination: Drone racing - First Person View (FPV)-bR4Gq9qfpnM.mp4
[download] 100% of 24.48MiB in 00:0079MiB/s ETA 00:00known ETA
